In [3]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, merge
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
import urllib
import collections
import os
import zipfile
import numpy as np
import tensorflow as tf
import numpy as np
import tensorflow as tf
import helpers
tf.reset_default_graph()
sess = tf.InteractiveSession()
import sys
import json
import re
import collections
import os
import datetime
import helpers

Provide the playlist file to create the embeddings

In [4]:
fullpath = 'mpd.slice.1000-1999.json'

In [ ]:
# Create the dataset for the embeddings
f = open(fullpath)
file_content = f.read()
f.close()
mpd_slice = json.loads(file_content)

# Input to seq to seq model
X_train = []

# Playlist name list
play_list_names = [re.sub('[^a-zA-Z0-9]+', '', mpd_slice['playlists'][idx]['name'].lower()) for idx in range(0, len(mpd_slice['playlists']))]

# Dictionary to maintain the song index and track name, other features can be added later
track_dict = {}
count  = len(set(play_list_names))
count += 2

# Looping to create the X_train
for idx in range(0, len(mpd_slice['playlists'])):
    lst_current = [] 
    
    lst_current.append(play_list_names.index(re.sub('[^a-zA-Z0-9]+', '', mpd_slice['playlists'][idx]['name'].lower())) + 2)    

    for idy in range(0, len(mpd_slice['playlists'][idx]['tracks'])):
        track_uri = re.sub('spotify:track:', '', mpd_slice['playlists'][idx]['tracks'][idy]['track_uri'])
        if(track_uri not in track_dict):
            track_dict[track_uri] = [count, mpd_slice['playlists'][idx]['tracks'][idy]['track_uri']]
            count += 1
        lst_current.append(track_dict[track_uri][0])
    X_train.append(lst_current)

print (X_train[0:100])
print (count)
print (len(play_list_names))
vocab = []
for x in X_train:
    vocab.extend(x)
    
print (vocab)


In [5]:
def maybe_download(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

In [6]:
# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

In [7]:
def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [8]:
def collect_data(vocabulary_size=10000):
    url = 'http://mattmahoney.net/dc/'
    filename = maybe_download('text8.zip', url, 31344016)
    #vocabulary = read_data(filename)
    vocabulary = vocab
    #print(vocabulary[:100])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary

In [9]:
vocab_size = count
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocab_size)
print(data[:7])

window_size = 20
vector_dim = 300
epochs = 200000

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

('Found and verified', 'text8.zip')
[1, 1281, 101, 1739, 6461, 1946, 343]


In [10]:
sampling_table = sequence.make_sampling_table(vocab_size)
couples, labels = skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print("Printing coiples", couples[:10], labels[:10])

('Printing coiples', [[22608, 1512], [3214, 3196], [38271, 95], [42040, 25473], [42051, 29381], [27042, 39212], [2233, 50720], [1600, 15009], [1360, 18425], [1826, 50727]], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1])


In [11]:
# create some input variables
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

# setup a cosine similarity operation which will be output in a secondary model
similarity = merge([target, context], mode='cos', dot_axes=0)

# now perform the dot product operation to get a similarity measure
dot_product = merge([target, context], mode='dot', dot_axes=1)
dot_product = Reshape((1,))(dot_product)
# add the sigmoid output layer
output = Dense(1, activation='sigmoid')(dot_product)
# create the primary training model
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

# create a secondary validation model to run our similarity checks during training
validation_model = Model(input=[input_target, input_context], output=similarity)

/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/ipykernel/__main__.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/ipykernel/__main__.py:15: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`
/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/ipykernel/

In [12]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()

In [ ]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 100 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    if cnt % 40000 == 0:
        sim_cb.run_sim()

In [14]:
#print("Printing coiples", couples[:10], labels[:10])

In [14]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()


In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 300)       17758500    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 300, 1)       0           embedding[0][0]                  
__________

In [16]:
wt = embedding.get_weights()
s = wt[0]
print (s.shape)

(59195, 300)


In [17]:
np.save("embedding.npy", s)